### Trade Demo

#### Goals:
- Login to the Canada domain.
- Select the dataset.
- Cacluate the sum of total of good imported to Egypt.
- Publish the result
- Download the results

### Step 1: Login into the Canada domain

In [1]:
%load_ext autoreload
%autoreload 2

In [29]:
# As a Data Scientist we want to perform some analysis on the trade dataset available on the Canada domain.
# Our goal is to calculate the sum total of the goods imported (imports + re-imports) to Egypt

import syft as sy

# Let's login into the canada domain
# canada_domain_node = sy.login(email="sheldon@caltech.edu", password="bazinga", port=8081)
canada_domain_node = sy.login(email="info@openmined.org", password="changethis", port=8081)

Connecting to http://localhost:8081... done! 	 Logging into canada... done!


### Step 2: Select the dataset

In [34]:
# Awesome !!! We're logged into the domain node
# Let's quickly list all the datasets available on the Canada domain

canada_domain_node.datasets

,data,tags,name,id,description
0,[],[],Canada Trade Data - First 10000 rows,fdd27bdd-e98c-4f82-b74d-0ab4ececc5e8,A collection of reports from Canada's statisti...
1,"[{'name': 'feb2020', 'id': '8d495cc1-c673-4b20...",[],Canada Trade Data - First 10000 rows,d1aff03e-4571-4e97-8c49-ae220124967d,A collection of reports from Canada's statisti...
2,"[{'name': 'feb2020', 'id': '13932405-6d7f-4c03...",[],Canada Trade Data - First 10000 rows,c424f8b9-5c4b-49fb-9785-31a6028d11d2,A collection of reports from Canada's statisti...


In [40]:
feb2020 = canada_domain_node.datasets[-1]['feb2020']

In [6]:
# Let's try to get the whole dataset itself.
feb2020.request(reason="Access whole dataset")

#### STOP: Return to Data Owner-Canada.ipynb - STEP 8!!

### Step 3: Try to download the whole dataset

In [41]:
# Let's see if our request was accepted
out = feb2020.publish(client=canada_domain_node ,sigma=1.0)

a
b
c
d


In [42]:
out.get()

[2021-08-13T13:04:01.602026+0100][CRITICAL][logger]][62221] UnknownPrivateException has been triggered.


UnknownPrivateException: UnknownPrivateException has been triggered.

In [ ]:
# Seems, like the request to download the whole dataset was denied by the Data Owner.

### Step 4: Private - Public Mutiplication

# dataset_columns = [
    "Classification",
    "Year",
    "Period",
    "Period Desc.",
    "Aggregate Level",
    "Is Leaf Code",
    "Trade Flow Code",
    "Trade Flow",
    "Reporter Code",
    "Reporter",
    "Reporter ISO",
    "Partner Code",
    "Partner",
    "Partner ISO",
    "Commodity Code",
    "Commodity",
    "Qty Unit Code",
    "Qty Unit",
    "Qty",
    "Netweight (kg)",
    "Trade Value (US$)",
    "Flag",
]

# Since the columns are represented by indicies in the private tensor,
# let's create a column to index map.
column_to_index_map = {c: i for i, c in enumerate(dataset_columns)}
column_to_index_map

In [17]:
# Let's calculate the value of the goods imported from Egypt.
# Patner Code for Egypt is : 818
partner_code_for_egypt = 818
trade_flow_codes_map = {
    "imports": 1,
    "exports": 2,
    "re-exports":3,
    "re-imports":4,
}

In [18]:
# Breaking it down
partner_codes = feb2020[:, column_to_index_map["Partner Code"]]
trade_flow_code = feb2020[:, column_to_index_map["Trade Flow Code"]]

egypt_mask = partner_codes == partner_code_for_egypt
imports_mask = trade_flow_code == trade_flow_codes_map["imports"]



In [26]:
# Let's calculate the good imported from Egypt
value_of_goods_imported_from_egypt_mask = egypt_mask * imports_mask

In [20]:
out = value_of_goods_imported_from_egypt_mask.publish(client=canada_domain_node ,sigma=1.0)

a
b
c
d


In [24]:
out.get()

[2021-08-13T13:00:40.967993+0100][CRITICAL][logger]][62221] UnknownPrivateException has been triggered.


UnknownPrivateException: UnknownPrivateException has been triggered.

In [12]:
# Let's continue with the analysis remotely.
# Let's calculate the value of the goods import from Egypt.

value_of_goods_reimported_from_egypt_mask = (
    feb2020[:, column_to_index_map["Partner Code"]] == partner_code_for_egypt
) * (feb2020[:, column_to_index_map["Trade Flow Code"]] == trade_flow_codes_map["re-imports"])

# Selecting Trade values for imports and re-imports (Private - Private Multiplication)
value_of_goods_imported_from_egypt = feb2020[:, column_to_index_map["Trade Value (US$)"]] * value_of_goods_imported_from_egypt_mask
value_of_goods_reimported_from_egypt = feb2020[:, column_to_index_map["Trade Value (US$)"]] * value_of_goods_reimported_from_egypt_mask

# Let's scale the trade value by 1000 - (Private - Public Multiplication)
scaled_value_of_goods_imported_from_egypt = value_of_goods_imported_from_egypt * (0.001)
scaled_value_of_goods_reimported_from_egypt = value_of_goods_reimported_from_egypt * (0.001)

### Step 5: Private - Private Addition

In [14]:
# sigma -> Amount of noise added to the response.
# Larger the sigma, smaller the budget spent.

# Calculating th total imported goods (Private - Private Addition)
private_total_imported_goods_value = scaled_value_of_goods_imported_from_egypt + scaled_value_of_goods_reimported_from_egypt
private_total_imported_goods_value = private_total_imported_goods_value.sum()

In [17]:
public_total_imported_goods_value = private_total_imported_goods_value.publish(client=canada_domain_node ,sigma=1.0)

a
b
c
d


In [ ]:
print(private_total_imported_goods_value.id_at_location)
print(public_total_imported_goods_value.id_at_location)

In [ ]:
private_total_imported_goods_value.update_searchability(pointable=True)

### Step 6: Download Results

In [ ]:
# Let's download the results
canada_domain_node.store

In [16]:
total_imported_goods_value = public_total_imported_goods_value.get()
print(f"Total trade value($) of the goods Imported from Egypt to Canada: ${total_imported_goods_value}")

[2021-08-12T17:02:29.530367-0400][CRITICAL][logger]][46552] UnknownPrivateException has been triggered.


UnknownPrivateException: UnknownPrivateException has been triggered.

In [ ]:
# b65f0512fd814ec58520b9cf009cada1>

*Great !!! We successfully, finished our analysis.*

# CONGRATULATIONS!!!

You've finished the demo!